In [51]:
import sqlite3
import pandas as pd
import numpy as np
import re
from datetime import date
from datetime import datetime

df = sqlite3.connect('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/raw/Miguel318.db')

dfPI = pd.read_sql_query("SELECT * FROM personal_info", df)


#Wrangling personal_info
    #Drop Unnamed:0 and lastName, I delete lastname column since we already have the full name and surname in rank_info
    
dfPI.drop(['Unnamed: 0'], axis='columns', inplace=True)
dfPI.drop(['lastName'], axis='columns', inplace=True)


    #Wrangling gender column
    
dfPI.gender.replace('[M]+$', 'Male', regex=True, inplace=True)
dfPI.gender.replace('[F]+$', 'Female', regex=True, inplace=True)
dfPI['gender'] = dfPI['gender'].astype('object').astype('str')
#print(dfPI['gender'].value_counts())
   
    #Wrangling age column
        #Correct info in age column: delete "years old" and change data types
        
dfPI.age.replace('[years old]+$', '', regex=True, inplace=True)   
dfPI['age'] = dfPI['age'].fillna(0)   #replace NaN values with 0.0
dfPI['age'] = dfPI['age'].astype('float').astype('int')    
#print(dfPI['age'].value_counts())    
#print(dfPI['age'].dtype)  

#Checking Types: BORRAR DESPUES

#print(dfPI['id'].dtype) / int64
#print(dfPI['age'].dtype) / int64 

#Datetime - preparing 

now = datetime.now()
now.year

    #Checking Outliers: 

stats = dfPI.describe().T
stats['IQR'] = stats['75%'] - stats['25%']
stats

outliers = pd.DataFrame(columns=dfPI.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = dfPI[(dfPI[col] < lower) | 
                   (dfPI[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)
    'sort=True'
    
outliers



,Outlier,age,country,gender,id,image
4,age,1985,None,Male,8242,https://specials-images.forbesimg.com/imageser...
42,age,1983,None,None,4838,https://specials-images.forbesimg.com/imageser...
54,age,0,None,Male,3034,https://specials-images.forbesimg.com/imageser...
76,age,0,Hong Kong,None,5052,https://specials-images.forbesimg.com/imageser...
84,age,1988,United States,Male,8484,https://specials-images.forbesimg.com/imageser...
...,...,...,...,...,...,...
2117,age,0,None,None,7723,https://specials-images.forbesimg.com/imageser...
2135,age,1992,None,Male,3766,https:http://specials-images.forbesimg.com/ima...
2136,age,1990,Ireland,Male,4114,https:http://specials-images.forbesimg.com/ima...
2154,age,1986,None,Male,6171,https://specials-images.forbesimg.com/imageser...
